### 📐K-최근접 이웃 알고리즘(K-Nearest Neighbor, KNN)
- 임의의 데이터가 주어지면 그 주변(이웃)의 데이터를 살펴본 뒤 더 많은 데이터가 포함되어 있는 범주로 분류하는 방식이다.
- 가장 간단한 머신러닝 알고리즘으로서, 직관적이고 나름 성능도 괜찮다.
- K를 어떻게 정하냐에 따라 결과 값이 바뀔 수 있다. K는 임의의 데이터가 주어졌을 때 가까운 이웃들의 개수이며, default값은 5이다.  
- K는 가장 가까운 5개의 이웃 데이터를 기반으로 분류하며, 일반적으로 홀수를 사용한다. 짝수일 경우 동점이 되어 하나의 결과를 도출할 수 없기 때문이다.
<div style="width: 80%; display: flex; margin-left: -20px; margin-bottom: 20px;">
    <div>
        <img src="./images/knn01.png">
    </div>
    <div>
        <img src="./images/knn02.png">
    </div>
</div>
- KNN은 fit을 통해 훈련을 시키면, 학습하지 않고 저장만 해놓기 때문에, Lazy Model이라 부른다.
- 새로운 데이터가 주어지면 그제서야 이웃 데이터를 보고 분류해나간다. 따라서 사전 모델링이 필요 없는 real-time 예측이 이루어진다.
- 미리 선을 그어놓거나 나눌 생각이 없기 때문에, SVM이나 선형 회귀보다 빠르다.
<img src="./images/knn03.jpg" width="350" style="margin:10px; margin-left: 0px">
- 데이터와 데이터 사이의 거리를 구해야 더 가까운 클래스로 분류할 수 있으며,  
이는 유클리드 거리(Euclidean Distance)방식과 맨해튼 거리(Manhattan Distance) 방식이 있다(/data-analysis/a_Numpy 에 있는 노름(Norm) 참조).